# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
pip install azureml-train-automl-runtime==1.22.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml'

experiment=Experiment(ws, experiment_name)


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ED5C9CL5P to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

csv_file = 'https://raw.githubusercontent.com/SwapnaKategaru/Project3/main/heart_failure_clinical_records_dataset.csv'

data = TabularDatasetFactory.from_delimited_files(csv_file)

In [7]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- numpy=1.16.0
- pip:
  - scikit-learn=0.22.1
  - azureml-defaults

Writing conda_dependencies.yml


In [8]:
# %%writefile train.py

from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files('https://raw.githubusercontent.com/SwapnaKategaru/Project3/main/heart_failure_clinical_records_dataset.csv')


def clean_data(data):
    # Dict for cleaning data
    df = data.to_pandas_dataframe()

    dataframe = pd.DataFrame()

    # dataframe = data.to_pandas_dataframe()
    dataframe[['age', 'creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time']] = df[['age', 'creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time']]
    normalized_data=(dataframe-dataframe.min())/(dataframe.max()-dataframe.min())
    normalized_data[['anaemia', 'diabetes','high_blood_pressure','sex','smoking']] = df[['anaemia', 'diabetes','high_blood_pressure','sex','smoking']]
    target_column = pd.DataFrame()
    target_column['DEATH_EVENT'] = df['DEATH_EVENT']
    target_column = target_column.squeeze()
    trained_T, Death_D = normalized_data, target_column
    return trained_T, Death_D

In [9]:
# from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
x, y = clean_data(data)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data = pd.concat([x_train, y_train], axis = 1)

In [10]:
from azureml.train.automl import AutoMLConfig
import logging

automl_settings = {
    "enable_early_stopping" : True,
    "max_concurrent_iterations": 4,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    training_data=train_data,
    label_column_name='DEATH_EVENT',
    n_cross_validations=2)

In [11]:
from azureml.widgets import RunDetails
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_4f25edac-9cc6-4736-af67-0c39ac9ca2f7

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

**************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [50]:
best_run, fitted_model = remote_run.get_output()

In [51]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/173e30fe/classes.interpret.json',
 'explanation/173e30fe/eval_data_viz.interpret.json',
 'explanation/173e30fe/expected_values.interpret.json',
 'explanation/173e30fe/features.interpret.json',
 'explanation/173e30fe/global_names/0.interpret.json',
 'explanation/173e30fe/global_rank/0.interpret.json',
 'explanation/173e30fe/global_values/0.interpret.json',
 'explanation/173e30fe/local_importance_values.interpret.json',
 'explanation/173e30fe/per_class_names/0.interpret.json',
 'explanation/173e30fe/per_class_rank/0.interpret.json',
 'explanation/173e30fe/per_class_values/0.interpret.json',
 'explanation/173e30fe/rich_metadata.interpret.json',
 'explanation/173e30fe/visualization_dict.interpret.json',
 'explanation/173e30fe/ys_pred_proba_viz.interpret.json',
 'explanation/173e30fe/ys_pred_viz.interpret.json',
 'explanation/d86ada26/classes.interpret.json',
 'explanation/d86ada26/eval_data_viz.interpret.json',
 'explanation/d86ada26/expected_values.interpret.json',
 'explanation/d86ada26/features.interpret.json',
 'explanation/d86ada26/global_names/0.interpret.json',
 'explanation/d86ada26/global_rank/0.interpret.json',
 'explanation/d86ada26/global_values/0.interpret.json',
 'explanation/d86ada26/local_importance_values.interpret.json',
 'explanation/d86ada26/per_class_names/0.interpret.json',
 'explanation/d86ada26/per_class_rank/0.interpret.json',
 'explanation/d86ada26/per_class_values/0.interpret.json',
 'explanation/d86ada26/rich_metadata.interpret.json',
 'explanation/d86ada26/visualization_dict.interpret.json',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [52]:
best_run.get_file_names()[-3]

'outputs/model.pkl'

In [53]:
import pickle

In [54]:
best_run.download_file('outputs/model.pkl', './outputs/automlmodel.pkl')

In [55]:
import joblib
joblib.load(filename='outputs/automlmodel.pkl')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_samples=None,
                                                                                                    min_impurity_decrease=0.0,
                                                                                                    min_impurity_split=None,
                      

In [56]:
from azureml.core.model import Model
model = Model.register(model_name='automl-model', model_path='outputs/automlmodel.pkl', workspace=ws)

Registering model automl-model


In [57]:
import sklearn

from azureml.core.environment import Environment

# environment = Environment("LocalDeploy")
environment = Environment.get(workspace=ws, name="AzureML-AutoML")

environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [29]:
# %%writefile score.py

import json
import numpy as np
import pandas as pd
import os
import joblib
import pickle

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automlmodel.pkl')
    print("Found model:", os.path.isfile(model_path)) #To check whether the model is actually present on the location we are looking at
    model = joblib.load(model_path)
def run(data):
    try:
        data = json.loads(data)['data']
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Writing score.py


In [60]:
pip install inference-schema

  Created wheel for wrapt: filename=wrapt-1.11.1-cp36-cp36m-linux_x86_64.whl size=66568 sha256=dc555869bae724742a3086ce36b797a4f962aec9399724c64358b0e79a5ac85a
  Stored in directory: /home/azureuser/.cache/pip/wheels/94/0f/ec/66085641573800014bb0c8b657f3366eff641c42df79abbfe9
Successfully built wrapt
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.2 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.2 which is incompatible.
ERROR: autokeras 1.0.12 has requirement tensorflow>=2.3.0, but you'll have tensorflow 2.1.0 which is incompatible.
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.12.1
    Uninstalling wrapt-1.12.1:
      Successfully uninstalled wrapt-1.12.1
Note: you may need to restart the kernel to use updated packages.


In [69]:
%%writefile score.py

import json
import numpy as np
import pandas as pd
import os
import joblib
import pickle

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

input_sample = pd.DataFrame({"age": pd.Series([0.0], dtype="float64"), "anaemia": pd.Series([0], dtype="int64"), "creatinine_phosphokinase": pd.Series([0.0], dtype="float64"), "diabetes": pd.Series([0], dtype="int64"), "ejection_fraction": pd.Series([0.0], dtype="float64"), "high_blood_pressure": pd.Series([0], dtype="int64"), "platelets": pd.Series([0.0], dtype="float64"), "serum_creatinine": pd.Series([0.0], dtype="float64"), "serum_sodium": pd.Series([0.0], dtype="float64"), "sex": pd.Series([0], dtype="int64"), "smoking": pd.Series([0], dtype="int64"), "time": pd.Series([0.0], dtype="float64")})
output_sample = np.array([0])


def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automlmodel.pkl')
    print("Found model:", os.path.isfile(model_path)) #To check whether the model is actually present on the location we are looking at
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))

def run(data):
    try:
        data = json.loads(data)['data']
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error        

Overwriting score.py


In [88]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-139731', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-139731'), name=automl-model, id=automl-model:1, version=1, tags={}, properties={})

In [89]:
Model(ws, 'automl-model')

Model(workspace=Workspace.create(name='quick-starts-ws-139731', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-139731'), name=automl-model, id=automl-model:1, version=1, tags={}, properties={})

In [91]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,  enable_app_insights=True, auth_enabled=True)

from azureml.core.webservice import LocalWebservice, Webservice
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=environment)
# deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, "test3", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://d3c13110-60ba-49a5-9888-d9eda056b386.southcentralus.azurecontainer.io/score
http://d3c13110-60ba-49a5-9888-d9eda056b386.southcentralus.azurecontainer.io/swagger.json


In [92]:
primary, secondary = service.get_keys()
print(primary)

6k9FfXS5JHrLkswumRlve8iKAlO9NOkC


In [94]:
import requests
import json

scoring_uri = 'http://d3c13110-60ba-49a5-9888-d9eda056b386.southcentralus.azurecontainer.io/score'

key = '6k9FfXS5JHrLkswumRlve8iKAlO9NOkC'


# Sample data to send to the service
data = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)
                

headers = {'Content-Type': 'application/json'}

headers['Authorization'] = f'Bearer {key}'

resp = requests.post(scoring_uri, data, headers=headers)
print(resp.text)

"{\"result\": [0, 0]}"


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=environment) 

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=1234)

local_service = Model.deploy(ws, "deeploy", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

In [40]:
print('Local service port: {}'.format(local_service.port))

Local service port: 1234


In [41]:
import json

sample_input = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)

local_service.run(sample_input)

[0, 0]

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service